In [103]:
# Easy way to handle adding geckodriver to PATH
from selenium import webdriver;
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager

import time
import wget
import os
browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())

root_path="https://fb.com/"
data_path="data/"



====== WebDriver manager ======
Current firefox version is 94.0
Get LATEST geckodriver version for 94.0 firefox
Driver [/home/lighthouse/.wdm/drivers/geckodriver/linux64/v0.30.0/geckodriver] found in cache
/tmp/ipykernel_6938/564049518.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [104]:
# Login to facebook
# Implementation Note: besure to slow down the iteractions between selenium and facebook, or facebook will detect the abnomal behavior
browser.get(root_path)

# Locating the username field
username = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.NAME, 'email')))
password = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.NAME, 'pass')))                                           

# Input the email and password
username.send_keys('taikhoanclone2411@gmail.com')
password.send_keys('thisisademopassword')
time.sleep(3)
username.submit()

# For this account, fb will ask for providing phone number as a safety measure
try:
    not_now =  WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.NAME, 'clicked_not_now[1]')))
    time.sleep(2)
    not_now.click()
except:
    pass

In [105]:
# We will download all images that have `alt` with string 'person' or 'people'
# For demo, we will only download 20 images

groups_ids = ['1953355488226190'] # We will hard-code the ids belonging to the groups from whose images we download from 
n = 10

for id in groups_ids:
    path_to_photos = '/groups/%s/media/photos'%id
    partial_path_to_photo = 'https://www.facebook.com/photo/'
    print(partial_path_to_photo)
    browser.get(root_path + path_to_photos)
    time.sleep(5)
    # Get the anchors DOM containing url that locating the photo resource, as the /photo/ path only show thumnails 
    anchors =  browser.find_elements(By.TAG_NAME, "a")        
    anchors_hrefs = [anchor.get_attribute("href") or "" for anchor in anchors] # get all anchors in DOM
    photos_hrefs = [href for href in anchors_hrefs if href.startswith(partial_path_to_photo)] # get anchors of images
    imgs = []
    browser.get(photos_hrefs[1]) # skip the first element as it is the group wallpaper
    
    for i in range(n):
        img = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "img[data-visualcompletion='media-vc-image']"))
        )
        alt = img.get_attribute('alt')
        if alt or False:
            if 'person' in alt or 'people' in alt:
                imgs.append(img.get_attribute('src'))
        time.sleep(3)
        next_button_section = browser.find_element(By.CSS_SELECTOR, "div[aria-label='Next photo']")
        hover = ActionChains(browser).move_to_element(next_button_section) # hover to the element, after this the element is changed
        next_button = browser.find_element(By.CSS_SELECTOR, "div[aria-label='Next photo']")  
        hover.perform()
        if not next_button:
            break
        else:
            next_button.click()
            # hover.click(next_button)
            # time.sleep(1)
            # hover.perform()
        
    for i, img in enumerate(imgs):
        save_as = data_path + str(i) + '.jpg'
        if os.path.exists(save_as):
            os.remove(save_as)
        wget.download(img, save_as)

https://www.facebook.com/photo/
100% [........................................................] 553011 / 553011